In [1]:
#IMPORTAR BIBLIOTECAS UTILIZADAS

import numpy as np
import pandas as pd
import random
from collections import Counter
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
"""
    Classe para processar o dataset
    Obs: o dataset tem que ser para problemas de classificacao e nele tem que conter uma coluna Class
"""
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

In [3]:
#IMPORTANDO DATASET

dataset = ProcessarDados('../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv')

In [4]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada

def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    #resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        #SEPARAR DADOS DE TREINO E TESTE MANTENDO A MESMA PROPORÇÃO  (80% E 20%)
        
        X_treino, X_teste, y_treino, y_teste = dataset.holdout(0.2)
        
        #ALTERANDO DIMENSÃO DE Y PARA D=1

        y_treino = y_treino.reshape(480,)
        y_teste = y_teste.reshape(120,)
                
        #ADICIONAR ATRIBUTOS IGUAIS A 1 EM X0

        X_treino = np.c_[np.ones(X_treino.shape[0]), X_treino]

        X_teste = np.c_[np.ones(X_teste.shape[0]), X_teste]
        
        #DEFINIR FUNCAO LOGISTICA (SIGMOID) PARA PREVER A PROBABILIDADE 

        def sigmoid(w,X):
            z = np.dot(X,w)
            return 1 / (1 + np.exp(-z))
        
        #DEFINIR FUNCAO PARA CLASSIFICAR
        
        def classificar(y, limiar=0.5):
            return (y > limiar)*1

        #DEFINIR FUNCAO CUSTO UTILIZANDO ENTROPIA CRUZADA

        def entropiaCruzada(previsto,y):
            return (-y * np.log(previsto) - (1 - y) * np.log(1 - previsto)).mean()
        
        #DEFINIR FUNCAO GRADIENTE DESCENDENTE PARA ATUALIZAR OS PARAMÊTROS w

        def gradienteDescendente(w, X, y, alpha):
            previsto = sigmoid(w,X)
            erro = previsto - y
            gradiente = np.dot(X.T, erro) / len(X)
            w -= alpha*gradiente

        #INICIALIZAR W (THETA) DE FORMA RANDOMICA e OTIMIZAR PARÂMETROS 

        w = np.random.rand(X_treino.shape[1])
        
        for i in range(3000):
            custo = entropiaCruzada(sigmoid(w,X_treino),y_treino)
            
            gradienteDescendente(w,X_treino,y_treino,0.01)
        
        #TREINANDO O MODELO
        
        classificados = classificar(sigmoid(w,X_treino))
        
        #TESTANDO O MODELO
        
        y_pred = classificar(sigmoid(w,X_teste))
        
        if exibir_matriz_confusao:
            print(confusion_matrix(y_teste, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_teste, y_pred)
        precision = metrics.precision_score(y_teste, y_pred)
        recall = metrics.recall_score(y_teste, y_pred)
        f1_score = 2 * (precision * recall) / (precision + recall)
        

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)


        if exibir_metricas:
            print("Rodada: #",i)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    
    return resultados_gerais

In [5]:
#TABELA COM RESULTADO DAS MÉTRICAS

def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False):
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")

In [6]:
#DEFININDO OS PARÂMETROS

epocas = 50
k_folds = 5
exibir_matriz_confusao = False
exibir_metricas = False

# TREINANDO O MODELO E OBTENDO AS MÉTRICAS

dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('REGRESSÃO LOGÍSTICA',dict_metricas)

============================================== REGRESSÃO LOGÍSTICA =================================================
=================================== TABELA DE MÉTRICAS DO MODELO ===================================
	 Accuracy 	|	 Precision 	|	 Recall 	|	 F1-Score
      0.90 +- 0.02             0.92 +- 0.03          0.94 +- 0.02             0.93 +- 0.02
